In [1]:
try:
    import json
except ImportError:
    import simplejson as json

# Import the necessary methods from "twitter" library
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream

In [2]:
import pandas as pd
# Obtain list of 50 most followed people worldwide from wikipedia
import wikipedia as wp
 
#Get the html source
#html = wp.page("List of most-followed Twitter accounts").html().encode("UTF-8")
#df = pd.read_html(html)[0]
#df.to_csv('twitter_list_of_influencers.csv',header=0,index=False)
df = pd.read_csv('twitter_list_of_influencers.csv')
user_list = list(df['Account name'])[:]
# remove '@' infront of the screen names and save the list as user_list
user_list = [a[1:] for a in user_list]

In [3]:
print(df.head(5))

   Rank  Change (monthly)    Account name          Owner  \
0   1.0               NaN      @katyperry     Katy Perry   
1   2.0               NaN   @justinbieber  Justin Bieber   
2   3.0               NaN    @BarackObama   Barack Obama   
3   4.0               NaN        @rihanna        Rihanna   
4   5.0               NaN  @taylorswift13   Taylor Swift   

   Followers (millions)               Activity Country  
0                   110               Musician     USA  
1                   107               Musician     CAN  
2                   103  Former U.S. President     USA  
3                    89               Musician     BAR  
4                    86               Musician     USA  


In [4]:
# Variables that contains the user credentials to access Twitter API 
ACCESS_TOKEN = '187549975-3LD41YaLCw3XnOvRUJVkqXrjt6gMsuT1HrEUlqDi'
ACCESS_SECRET = 'ZykwgJpFYQaZdP6vEFHtbqBfkQfgBo9mV0LM3MkmEp5Oj'
CONSUMER_KEY = 'qQTzJ4OceyUFMLcWkn7ZZ5Wrp'
CONSUMER_SECRET = 'z6aV6zjg2yac1TiZw5ERUvZ2XLXAGBQJ1OD0yUCFtCczWNKjik'

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

t = Twitter(auth=oauth)
id_list = t.users.lookup(screen_name=','.join(user_list))

In [5]:
user_id = [id_list[x]['id_str'] for x in range(50)]
print(user_id)

['21447363', '27260086', '813286', '79293791', '17919972', '14230524', '15846407', '155659213', '10228272', '26565946', '783214', '25365536', '16409683', '34507480', '21111883', '23375688', '428333', '25073877', '44409004', '15485441', '50393960', '85603854', '18839785', '100220864', '19397785', '807095', '23083404', '268414482', '759251', '105119490', '158487331', '180505807', '5402612', '27195114', '101311381', '26257166', '23151437', '145125358', '116362700', '2557521', '20322929', '132385468', '84279963', '28706024', '158314798', '181561712', '209708391', '35094637', '14872237', '60865434']


In [6]:
user_info = pd.DataFrame({"user_name":user_list,"user_id":user_id,"activity":df['Activity'],"country":df['Country']})
user_info.head(10)
user_info.to_csv("user_info.csv")

In [7]:
# Initiate the connection to Twitter Streaming API
twitter_stream = TwitterStream(auth=oauth)
iterator = twitter_stream.statuses.filter(lang='en',follow=','.join(user_id))

# Get a sample of the public data following through Twitter
# iterator = twitter_stream.statuses.sample()
# As a pilot survey we set it to stop after getting 1000 tweets. 
# You don't have to set it to stop, but can continue running 
# the Twitter API to collect data for days or even longer. 

tweets_filename = 'twitter_savereplies_nsamp30000.json'
#tweets_file = open(tweets_filename, "r")
import os.path
# find out a way to filter replies only:
# save to json file
tweet_count = 30000
tweet_cnt = tweet_count
tweets = []
try:
    os.path.isfile(tweets_filename)
    with open(tweets_filename, 'r') as f:
        tweets = json.load(f) # readline only the first tweet/line
        print("file exists")
        if len(tweets) < tweet_cnt:
            raise DataError
        pass
except:
    tweet_count = tweet_cnt - len(tweets)
    print("file not complete, stream twitter for collecting further tweets")
    new_tweets = []
    for tweet in iterator:
    # select only "replies" to top 50 followed users
        try:
            if str(tweet['in_reply_to_user_id']) in user_id:
        #print(tweet['in_reply_to_user_id'])
                tweet_count -= 1
                new_tweets.append(tweet)
        except:
            continue
        if tweet_count <= 0:
            break 
    # Twitter Python Tool wraps the data returned by Twitter 
    # as a TwitterDictResponse object.
    # We convert it back to the JSON format to print/score
    # loads converts json format to python dictionary
    # dumps converts python dictionary to json format
    # The command below will do pretty printing for JSON data, try it out
    # print json.dumps(tweet, indent=4)
#        with open(tweets_filename, 'w') as outfile:
#            json.dump(tweets,outfile,indent=4)

#    tweets.update(new_tweets)
    tweets.extend(new_tweets)
    with open(tweets_filename, 'w') as f:
        json.dump(tweets, f)

file not complete, stream twitter for collecting further tweets


In [8]:
print(len(tweets))

30000
